In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.base.model import GenericLikelihoodModel
import scipy.stats as stats
import sys

sys.path.append("../")
import vuong_tests11 as vuong_tests_fast

In [2]:
class OLS_loglike(GenericLikelihoodModel):
    
    def __init__(self, *args,ols=False, **kwargs):
        super(OLS_loglike,self).__init__(*args,**kwargs)
        self.ols = ols

    def loglikeobs(self, params):
        y = self.endog
        x = self.exog
        mu_y = np.matmul(x,params)  
        resid = y - mu_y
        sigma = np.sqrt(np.sum(resid**2)/resid.shape[0])
        pr_y = stats.norm.logpdf( resid, loc=0,scale=sigma )
        return pr_y


def setup_shi(yn,xn,return_model=False,num_params=4):
    x1n,x2n = xn[:,0],xn[:,1:num_params+1]
    
    # model 1 grad, etc.
    model1 = sm.OLS(yn,sm.add_constant(x1n))
    model1_fit = model1.fit(disp=False)
    params1 = (model1_fit.params)
    
    model1_deriv = OLS_loglike(yn,sm.add_constant(x1n))
    ll1 = model1_deriv.loglikeobs(model1_fit.params)
    grad1 =  model1_deriv.score_obs(model1_fit.params)    
    hess1 = model1_deriv.hessian(model1_fit.params)
    
    #model 2 grad, etc.
    model2 = sm.OLS(yn,sm.add_constant(x2n))
    model2_fit = model2.fit(disp=False)
    params2 = (model2_fit.params)
    
    model2_deriv = OLS_loglike(yn,sm.add_constant(x2n))
    ll2 = model2_deriv.loglikeobs(model2_fit.params)
    grad2 =  model2_deriv.score_obs(model2_fit.params)    
    hess2 = model2_deriv.hessian(model2_fit.params)
    
    if return_model:
        return ll1,grad1,hess1,params1,model1,ll2,grad2,hess2,params2,model2
    return ll1,grad1,hess1,params1,ll2,grad2,hess2,params2


In [3]:
def gen_data(nobs=1000, a=0.25, num_params=4):
    x = np.random.normal(scale=1., size=(nobs,1+num_params))
    e = np.random.normal(loc=0.0, scale=1.0, size=nobs)
    y = 1 + a*x[:,0] + a/np.sqrt(num_params)*x[:,1:num_params+1].sum(axis=1) + e
    return y,x,nobs

yn,xn,nobs = gen_data()
ll1,grad1,hess1,params1,ll2,grad2,hess2,params2 = setup_shi(yn,xn,return_model=False,num_params=15)
print(grad1.shape,hess1.shape)

(1000, 2) (2, 2)


In [4]:
num_sims = 500
trials =1000
adapt_c = False
data_tuned_epsilon = True
epsilon = .5

size weirdness rn...

In [5]:
a = 0.25
num_params=9

setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=nobs, a=a,  num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=epsilon,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out,data_tuned_epsilon=data_tuned_epsilon)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & SW Test & Naive Bootstrap & Pairwise Bootstrap & Shi (2015) \\ \hline \hline
No selection & 0.92 & 0.92 & 0.96 & 0.78 & 0.96 & 0.95   \\
Model 1 & 0.01 & 0.01 & 0.01 & 0.14 & 0.02 & 0.02   \\
Model 2 & 0.06 & 0.06 & 0.03 & 0.08 & 0.02 & 0.03   \\
\hline
\end{tabular}
(array([0.924, 0.012, 0.064]), array([0.924, 0.012, 0.064]), array([0.94 , 0.024, 0.036]), array([0.78 , 0.138, 0.082]), array([0.938, 0.042, 0.02 ]), array([0.96 , 0.014, 0.026]), array([0.956, 0.02 , 0.024]), array([0.95 , 0.022, 0.028]), -3.8860307273092154, 11.648642969184259, 10.96897982632219)


# power 0.... need to see...

In [6]:
a1,a2 = np.sqrt(1.09-1), 0.00
num_params= 9


def gen_data2(nobs=1000, a1=np.sqrt(1.09-1), a2=0.00 , num_params=19):
    x = np.random.normal(scale=1., size=(nobs,1+num_params))
    e = np.random.normal(loc=0.0, scale=1.0, size=nobs)
    y = 1 + a1*x[:,0] + a2/np.sqrt(num_params)*x[:,1:num_params+1].sum(axis=1) + e
    return y,x,nobs

In [7]:
calc_c =lambda nobs: (nobs**(-1/2)/100,nobs**(1/5) )

nobs = 250

a1,a2 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=epsilon,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out,data_tuned_epsilon=data_tuned_epsilon)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & SW Test & Naive Bootstrap & Pairwise Bootstrap & Shi (2015) \\ \hline \hline
No selection & 0.83 & 0.84 & 0.91 & 0.50 & 0.88 & 0.54   \\
Model 1 & 0.17 & 0.16 & 0.09 & 0.49 & 0.11 & 0.46   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.832, 0.168, 0.   ]), array([0.842, 0.158, 0.   ]), array([0.71, 0.29, 0.  ]), array([0.502, 0.494, 0.004]), array([0.518, 0.482, 0.   ]), array([0.91 , 0.086, 0.004]), array([0.882, 0.114, 0.004]), array([0.54, 0.46, 0.  ]), 6.651673892425046, 5.192576079968966, 5.41508850176521)


In [8]:
nobs = 250

a1,a2 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=epsilon,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out,data_tuned_epsilon=data_tuned_epsilon)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & SW Test & Naive Bootstrap & Pairwise Bootstrap & Shi (2015) \\ \hline \hline
No selection & 0.81 & 0.83 & 0.92 & 0.52 & 0.88 & 0.56   \\
Model 1 & 0.19 & 0.17 & 0.07 & 0.47 & 0.11 & 0.44   \\
Model 2 & 0.00 & 0.00 & 0.01 & 0.00 & 0.01 & 0.00   \\
\hline
\end{tabular}
(array([0.814, 0.186, 0.   ]), array([0.826, 0.174, 0.   ]), array([0.746, 0.254, 0.   ]), array([0.524, 0.474, 0.002]), array([0.55, 0.45, 0.  ]), array([0.92 , 0.074, 0.006]), array([0.884, 0.11 , 0.006]), array([0.558, 0.442, 0.   ]), 6.540717242938334, 5.027886670075842, 5.328707725731636)


In [9]:
nobs = 250

a1,a2 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=epsilon,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out,data_tuned_epsilon=data_tuned_epsilon)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & SW Test & Naive Bootstrap & Pairwise Bootstrap & Shi (2015) \\ \hline \hline
No selection & 0.85 & 0.85 & 0.88 & 0.51 & 0.87 & 0.57   \\
Model 1 & 0.15 & 0.15 & 0.11 & 0.48 & 0.12 & 0.43   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.01 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.846, 0.154, 0.   ]), array([0.85, 0.15, 0.  ]), array([0.726, 0.274, 0.   ]), array([0.514, 0.48 , 0.006]), array([0.558, 0.442, 0.   ]), array([0.884, 0.112, 0.004]), array([0.874, 0.122, 0.004]), array([0.572, 0.428, 0.   ]), 6.449013176317757, 5.194449821716406, 5.335531039929717)


In [10]:
nobs = 250

a1,a2 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=epsilon,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out,data_tuned_epsilon=data_tuned_epsilon)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & SW Test & Naive Bootstrap & Pairwise Bootstrap & Shi (2015) \\ \hline \hline
No selection & 0.83 & 0.84 & 0.92 & 0.51 & 0.89 & 0.55   \\
Model 1 & 0.17 & 0.16 & 0.08 & 0.48 & 0.11 & 0.45   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.832, 0.168, 0.   ]), array([0.836, 0.164, 0.   ]), array([0.724, 0.276, 0.   ]), array([0.514, 0.482, 0.004]), array([0.516, 0.484, 0.   ]), array([0.92 , 0.078, 0.002]), array([0.892, 0.106, 0.002]), array([0.552, 0.448, 0.   ]), 6.795449519138317, 4.867821761090499, 5.3751380663084225)


In [11]:
data_tuned_c = .9
calc_c =lambda nobs: (nobs**(-1/2)/50,nobs**(1/10) )

nobs = 250

a1,a2 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=epsilon,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out,data_tuned_epsilon=data_tuned_epsilon)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & SW Test & Naive Bootstrap & Pairwise Bootstrap & Shi (2015) \\ \hline \hline
No selection & 0.83 & 0.83 & 0.91 & 0.53 & 0.88 & 0.58   \\
Model 1 & 0.17 & 0.17 & 0.09 & 0.46 & 0.11 & 0.42   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.01 & 0.01 & 0.00   \\
\hline
\end{tabular}
(array([0.828, 0.172, 0.   ]), array([0.832, 0.168, 0.   ]), array([0.74, 0.26, 0.  ]), array([0.532, 0.462, 0.006]), array([0.56, 0.44, 0.  ]), array([0.906, 0.09 , 0.004]), array([0.884, 0.11 , 0.006]), array([0.584, 0.416, 0.   ]), 6.40120419363725, 5.160527893722187, 5.298880021947743)


# a  = .25, k= 9

In [12]:
a = 0.25
num_params=9



In [13]:
nobs = 250

setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=250, a=a,  num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=epsilon,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out,data_tuned_epsilon=data_tuned_epsilon)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & SW Test & Naive Bootstrap & Pairwise Bootstrap & Shi (2015) \\ \hline \hline
No selection & 0.93 & 0.93 & 0.95 & 0.82 & 0.94 & 0.98   \\
Model 1 & 0.00 & 0.00 & 0.02 & 0.13 & 0.03 & 0.01   \\
Model 2 & 0.07 & 0.07 & 0.03 & 0.05 & 0.03 & 0.01   \\
\hline
\end{tabular}
(array([0.926, 0.004, 0.07 ]), array([0.928, 0.004, 0.068]), array([0.96 , 0.018, 0.022]), array([0.816, 0.134, 0.05 ]), array([0.934, 0.06 , 0.006]), array([0.952, 0.02 , 0.028]), array([0.942, 0.026, 0.032]), array([0.978, 0.012, 0.01 ]), -3.5906116754026587, 5.785306938518636, 6.001497122438215)


In [14]:
nobs = 500

setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=nobs, a=a,  num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=epsilon,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out,data_tuned_epsilon=data_tuned_epsilon)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & SW Test & Naive Bootstrap & Pairwise Bootstrap & Shi (2015) \\ \hline \hline
No selection & 0.94 & 0.94 & 0.95 & 0.86 & 0.94 & 0.97   \\
Model 1 & 0.01 & 0.01 & 0.02 & 0.10 & 0.03 & 0.02   \\
Model 2 & 0.05 & 0.05 & 0.03 & 0.05 & 0.03 & 0.01   \\
\hline
\end{tabular}
(array([0.94 , 0.006, 0.054]), array([0.94 , 0.006, 0.054]), array([0.966, 0.016, 0.018]), array([0.856, 0.098, 0.046]), array([0.942, 0.046, 0.012]), array([0.946, 0.024, 0.03 ]), array([0.942, 0.032, 0.026]), array([0.972, 0.016, 0.012]), -3.4025949143781538, 7.650863871381886, 8.03614997737624)


In [15]:
nobs = 100

setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=nobs, a=a,  num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=epsilon,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out,data_tuned_epsilon=data_tuned_epsilon)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & SW Test & Naive Bootstrap & Pairwise Bootstrap & Shi (2015) \\ \hline \hline
No selection & 0.87 & 0.96 & 0.94 & 0.83 & 0.92 & 0.99   \\
Model 1 & 0.00 & 0.00 & 0.01 & 0.13 & 0.03 & 0.01   \\
Model 2 & 0.13 & 0.04 & 0.05 & 0.03 & 0.05 & 0.01   \\
\hline
\end{tabular}
(array([0.874, 0.   , 0.126]), array([0.96, 0.  , 0.04]), array([0.964, 0.012, 0.024]), array([0.832, 0.134, 0.034]), array([0.936, 0.062, 0.002]), array([0.938, 0.012, 0.05 ]), array([0.924, 0.028, 0.048]), array([0.986, 0.006, 0.008]), -4.132143461182659, 4.0686433062680365, 4.354330376206858)


# a  = .25, k= 4, something not right?

In [16]:
a = 0.25
num_params=4



In [17]:
nobs = 250

setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=nobs, a=a,  num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=epsilon,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out,data_tuned_epsilon=data_tuned_epsilon)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & SW Test & Naive Bootstrap & Pairwise Bootstrap & Shi (2015) \\ \hline \hline
No selection & 0.96 & 0.96 & 0.94 & 0.82 & 0.92 & 0.98   \\
Model 1 & 0.01 & 0.01 & 0.02 & 0.10 & 0.03 & 0.01   \\
Model 2 & 0.03 & 0.03 & 0.05 & 0.09 & 0.05 & 0.01   \\
\hline
\end{tabular}
(array([0.964, 0.006, 0.03 ]), array([0.964, 0.006, 0.03 ]), array([0.962, 0.01 , 0.028]), array([0.816, 0.098, 0.086]), array([0.946, 0.034, 0.02 ]), array([0.936, 0.016, 0.048]), array([0.916, 0.03 , 0.054]), array([0.984, 0.008, 0.008]), -1.2590609190439819, 5.524481360861716, 5.660689149287663)


# a  = .25, K = 19

In [18]:
a = 0.25
num_params=19


In [19]:
nobs = 250

setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=250, a=a,  num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=epsilon,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out,data_tuned_epsilon=data_tuned_epsilon)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & SW Test & Naive Bootstrap & Pairwise Bootstrap & Shi (2015) \\ \hline \hline
No selection & 0.72 & 0.79 & 0.92 & 0.78 & 0.94 & 0.97   \\
Model 1 & 0.00 & 0.00 & 0.01 & 0.19 & 0.02 & 0.01   \\
Model 2 & 0.28 & 0.21 & 0.07 & 0.03 & 0.05 & 0.02   \\
\hline
\end{tabular}
(array([0.724, 0.   , 0.276]), array([0.79, 0.  , 0.21]), array([0.938, 0.014, 0.048]), array([0.78 , 0.186, 0.034]), array([0.878, 0.112, 0.01 ]), array([0.922, 0.012, 0.066]), array([0.936, 0.018, 0.046]), array([0.968, 0.012, 0.02 ]), -9.637058901291342, 6.649378634502686, 6.720400184771328)


# evidence of power

In [20]:
nobs = 500

a1,a2 = np.sqrt(1.09**.5-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=epsilon,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out,data_tuned_epsilon=data_tuned_epsilon)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & SW Test & Naive Bootstrap & Pairwise Bootstrap & Shi (2015) \\ \hline \hline
No selection & 0.98 & 0.98 & 0.93 & 0.56 & 0.92 & 0.65   \\
Model 1 & 0.02 & 0.02 & 0.05 & 0.42 & 0.05 & 0.35   \\
Model 2 & 0.00 & 0.00 & 0.02 & 0.02 & 0.02 & 0.00   \\
\hline
\end{tabular}
(array([0.978, 0.018, 0.004]), array([0.98 , 0.018, 0.002]), array([0.876, 0.124, 0.   ]), array([0.56 , 0.424, 0.016]), array([0.58, 0.42, 0.  ]), array([0.928, 0.048, 0.024]), array([0.924, 0.054, 0.022]), array([0.648, 0.352, 0.   ]), 1.3731555962790518, 5.495990398710595, 6.23642434125363)


In [21]:
nobs = 100

a1,a2 = np.sqrt(1.09**2.5-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=epsilon,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out,data_tuned_epsilon=data_tuned_epsilon)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & SW Test & Naive Bootstrap & Pairwise Bootstrap & Shi (2015) \\ \hline \hline
No selection & 0.94 & 0.99 & 0.93 & 0.38 & 0.87 & 0.67   \\
Model 1 & 0.04 & 0.01 & 0.05 & 0.62 & 0.11 & 0.33   \\
Model 2 & 0.02 & 0.00 & 0.02 & 0.00 & 0.01 & 0.00   \\
\hline
\end{tabular}
(array([0.942, 0.042, 0.016]), array([0.994, 0.006, 0.   ]), array([0.75, 0.25, 0.  ]), array([0.376, 0.624, 0.   ]), array([0.324, 0.676, 0.   ]), array([0.928, 0.054, 0.018]), array([0.872, 0.114, 0.014]), array([0.666, 0.334, 0.   ]), 0.8525983303093528, 6.013170014887639, 5.814245706491829)


In [22]:
nobs = 250

a1,a2 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=4)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=epsilon,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out,data_tuned_epsilon=data_tuned_epsilon)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & SW Test & Naive Bootstrap & Pairwise Bootstrap & Shi (2015) \\ \hline \hline
No selection & 0.58 & 0.58 & 0.85 & 0.53 & 0.84 & 0.53   \\
Model 1 & 0.42 & 0.42 & 0.15 & 0.46 & 0.15 & 0.47   \\
Model 2 & 0.00 & 0.00 & 0.01 & 0.01 & 0.01 & 0.00   \\
\hline
\end{tabular}
(array([0.584, 0.416, 0.   ]), array([0.584, 0.416, 0.   ]), array([0.67, 0.33, 0.  ]), array([0.53, 0.46, 0.01]), array([0.572, 0.428, 0.   ]), array([0.848, 0.146, 0.006]), array([0.842, 0.15 , 0.008]), array([0.532, 0.468, 0.   ]), 9.183351943228665, 4.613005570766531, 4.923345768403703)


In [23]:
nobs = 250

a1,a2 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=19)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=epsilon,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out,data_tuned_epsilon=data_tuned_epsilon)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & SW Test & Naive Bootstrap & Pairwise Bootstrap & Shi (2015) \\ \hline \hline
No selection & 0.96 & 0.98 & 0.95 & 0.45 & 0.91 & 0.64   \\
Model 1 & 0.03 & 0.02 & 0.04 & 0.55 & 0.08 & 0.36   \\
Model 2 & 0.01 & 0.00 & 0.01 & 0.00 & 0.01 & 0.00   \\
\hline
\end{tabular}
(array([0.964, 0.026, 0.01 ]), array([0.98 , 0.016, 0.004]), array([0.824, 0.174, 0.002]), array([0.446, 0.55 , 0.004]), array([0.424, 0.574, 0.002]), array([0.948, 0.042, 0.01 ]), array([0.91, 0.08, 0.01]), array([0.638, 0.362, 0.   ]), 1.5065924686758991, 5.594860304729026, 6.137575824761724)


# evidence of power 2

In [24]:
nobs = 250


a2,a1 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=epsilon,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out,data_tuned_epsilon=data_tuned_epsilon)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & SW Test & Naive Bootstrap & Pairwise Bootstrap & Shi (2015) \\ \hline \hline
No selection & 0.01 & 0.60 & 0.80 & 0.81 & 0.84 & 0.64   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.02 & 0.00 & 0.00   \\
Model 2 & 0.99 & 0.40 & 0.20 & 0.17 & 0.16 & 0.36   \\
\hline
\end{tabular}
(array([0.012, 0.   , 0.988]), array([0.596, 0.   , 0.404]), array([0.586, 0.   , 0.414]), array([0.814, 0.016, 0.17 ]), array([0.908, 0.002, 0.09 ]), array([0.8  , 0.002, 0.198]), array([0.84 , 0.002, 0.158]), array([0.64, 0.  , 0.36]), -20.433581397857395, 5.552736653214346, 6.159580374139919)


In [25]:
nobs = 500


a2,a1 = np.sqrt(1.09**.5-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=epsilon,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out,data_tuned_epsilon=data_tuned_epsilon)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & SW Test & Naive Bootstrap & Pairwise Bootstrap & Shi (2015) \\ \hline \hline
No selection & 0.01 & 0.35 & 0.88 & 0.77 & 0.88 & 0.67   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.05 & 0.00 & 0.00   \\
Model 2 & 0.99 & 0.65 & 0.12 & 0.18 & 0.11 & 0.33   \\
\hline
\end{tabular}
(array([0.006, 0.   , 0.994]), array([0.346, 0.   , 0.654]), array([0.746, 0.   , 0.254]), array([0.768, 0.05 , 0.182]), array([0.922, 0.   , 0.078]), array([0.882, 0.   , 0.118]), array([0.884, 0.002, 0.114]), array([0.672, 0.   , 0.328]), -20.367744387341528, 5.723968941090471, 6.3002986443277695)


In [26]:
nobs = 100


a2,a1 =np.sqrt(1.09**2.5-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=epsilon,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out,data_tuned_epsilon=data_tuned_epsilon)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & SW Test & Naive Bootstrap & Pairwise Bootstrap & Shi (2015) \\ \hline \hline
No selection & 0.00 & 0.95 & 0.59 & 0.78 & 0.67 & 0.54   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.01 & 0.00 & 0.00   \\
Model 2 & 1.00 & 0.05 & 0.41 & 0.21 & 0.33 & 0.46   \\
\hline
\end{tabular}
(array([0.002, 0.   , 0.998]), array([0.948, 0.   , 0.052]), array([0.428, 0.   , 0.572]), array([0.782, 0.01 , 0.208]), array([0.846, 0.   , 0.154]), array([0.592, 0.   , 0.408]), array([0.67, 0.  , 0.33]), array([0.544, 0.   , 0.456]), -21.049205348587073, 5.841280162811713, 5.793717296102239)


In [27]:
nobs = 250


a2,a1 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=250, a1=a1, a2=a2, num_params=4)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=epsilon,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out,data_tuned_epsilon=data_tuned_epsilon)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & SW Test & Naive Bootstrap & Pairwise Bootstrap & Shi (2015) \\ \hline \hline
No selection & 0.19 & 0.19 & 0.81 & 0.60 & 0.82 & 0.49   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.02 & 0.01 & 0.00   \\
Model 2 & 0.81 & 0.81 & 0.18 & 0.38 & 0.17 & 0.51   \\
\hline
\end{tabular}
(array([0.188, 0.   , 0.812]), array([0.19, 0.  , 0.81]), array([0.598, 0.   , 0.402]), array([0.6  , 0.016, 0.384]), array([0.662, 0.   , 0.338]), array([0.814, 0.004, 0.182]), array([0.822, 0.006, 0.172]), array([0.494, 0.   , 0.506]), -12.644669602648282, 4.851271868500527, 4.9500451400077035)


In [28]:
nobs = 250


a2,a1 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=19)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=epsilon,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out,data_tuned_epsilon=data_tuned_epsilon)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & SW Test & Naive Bootstrap & Pairwise Bootstrap & Shi (2015) \\ \hline \hline
No selection & 0.01 & 0.66 & 0.80 & 0.77 & 0.83 & 0.69   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.04 & 0.01 & 0.00   \\
Model 2 & 0.99 & 0.34 & 0.20 & 0.18 & 0.16 & 0.31   \\
\hline
\end{tabular}
(array([0.012, 0.   , 0.988]), array([0.66, 0.  , 0.34]), array([0.652, 0.   , 0.348]), array([0.774, 0.042, 0.184]), array([0.934, 0.006, 0.06 ]), array([0.804, 0.   , 0.196]), array([0.83 , 0.006, 0.164]), array([0.694, 0.   , 0.306]), -19.69300146511794, 5.536975619321841, 6.041196902022812)
